In [ ]:
# 1
# Chance Wiese

In [6]:
# 2
print('''
Name: Puptrail, Inc.
Product: “teardrop” camper trailers
Locations:
    1. Salt Lake City
    2. Phoenix
    3. Boise
''')


Company Name: Puptrail, Inc.
Company Product: “teardrop” camper trailers
Locations:
    1. Salt Lake City
    2. Phoenix
    3. Boise



In [35]:
# 3
import pandas as pd
import numpy as np
import psycopg2

psycopg2_conn = psycopg2.connect(
    database="puptrail", 
    user='dw_chancewiese',
    password='Spikeball2020',
    host='database-1.czsooswggscz.us-east-2.rds.amazonaws.com',
    port='5432'
)

psycopg2_cursor = psycopg2_conn.cursor()

query = '''
SELECT 
    (o.builtdate - o.promisedate) AS delay_days,
    COUNT(o.*) AS num_trailers,
    AVG(s.delsatisfac) AS avg_satisfaction
FROM oline o
JOIN salesorder s ON o.orderid = s.orderid
JOIN product p ON o.productid = p.prodid
JOIN prodcat pc ON p.catid = pc.catid
WHERE pc.category = 'Trailers'
AND o.builtdate > o.promisedate
GROUP BY delay_days
ORDER BY delay_days;
'''

df = pd.read_sql(query, psycopg2_conn)
print("Delay Info:")
print(df)

df_corr = df.corr()
print("\nCorrelation Matrix:")
print(df_corr)

print("\nRelationship:")
print('''There is a slight positive relationship between delay days and average satisfaction (correlation = 0.09).
This tells us that customers are generally understanding of delays and aren't too unhappy with them.
There are generally less trailers as delay days increase, which is good, but may also skew satisfaction.
Where there are about 8-14 days delayed, more trailers are present and average satisfaction does drop, 
so it should be noted that with more trailers (and reviews), satisfaction does drop with more delay days.''')

Delay Info:
    delay_days  num_trailers  avg_satisfaction
0            1             6          4.333333
1            2             6          3.250000
2            3            12          3.583333
3            4             9          3.571429
4            5             9          4.285714
5            6            14          3.500000
6            7             7          4.000000
7            8            17          3.642857
8            9            16          4.692308
9           10            13          2.666667
10          11            16          3.733333
11          12            11          3.818182
12          13             8          3.750000
13          14            13          3.818182
14          16             4          4.250000
15          17             2          3.000000
16          18             2          4.500000
17          19             3          4.000000

Correlation Matrix:
                  delay_days  num_trailers  avg_satisfaction
delay_days   

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_5348/1756534001.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [37]:
# 4
query = '''
WITH trailer_sales AS (
    SELECT 
        sp.spid,
        sp.spfname,
        sp.splname,
        si.sitename,
        COUNT(DISTINCT CASE WHEN pc.category = 'Trailers' THEN ol.orderid END) AS trailers_sold,
        COUNT(DISTINCT CASE WHEN pc.category = 'Upgrades' THEN ol.olid END) AS upgrades_sold,
        SUM(CASE WHEN pc.category = 'Upgrades' THEN ol.linetotal ELSE 0 END) AS upgrade_revenue
    FROM salesperson sp
    JOIN site si ON sp.siteid = si.siteid
    JOIN salesorder so ON sp.spid = so.sellerid
    JOIN oline ol ON so.orderid = ol.orderid
    JOIN product p ON ol.productid = p.prodid
    JOIN prodcat pc ON p.catid = pc.catid
    WHERE sp.termdate IS NULL
    GROUP BY sp.spid, sp.spfname, sp.splname, si.sitename
)
SELECT 
    spfname,
    splname,
    sitename,
    trailers_sold,
    upgrades_sold,
    upgrade_revenue,
    CAST(upgrade_revenue / trailers_sold AS NUMERIC(10,2)) AS avg_upsell_per_trailer
FROM trailer_sales
ORDER BY avg_upsell_per_trailer DESC;
'''

df = pd.read_sql(query, psycopg2_conn)
print("Upsell Info by Salesperson:")
print(df)

Upsell Info by Salesperson:
    spfname     splname        sitename  trailers_sold  upgrades_sold  \
0    Maryla    Kunibert  Salt Lake City            276            335   
1   Milburn       Guido  Salt Lake City            309            361   
2     Kelan      Fowler           Boise            206            212   
3    Shayan      Hewitt           Boise            291            320   
4   Alberto       Dimas  Salt Lake City            112            121   
5    Archie      Dinesh  Salt Lake City            171            175   
6   Roksana     Antonia           Boise            297            235   
7     Andre       Clare         Phoenix            200             93   
8     Robby        Lexy         Phoenix            141             54   
9    Bethel      Derick         Phoenix            131             46   
10     Miah  Wittington         Phoenix            131             31   

    upgrade_revenue  avg_upsell_per_trailer  
0          287825.0                 1042.84  
1  

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_5348/785358344.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [ ]:
# 5
import pymssql

pymssql_conn = pymssql.connect(
    server='stairway.usu.edu',
    user='5330user',
    password='pipelinesnow',
    database='pup_sl'
)

pymssql_conn_cursor = pymssql_conn_conn.cursor()